In [6]:
from ortools.sat.python import cp_model
import pandas as pd
from IPython.display import display

num_members = 40
num_shifts = 4
num_days = 100
all_members = range(num_members)
all_shifts = range(num_shifts)
all_days = range(num_days)
starting_day = 0 #day of week mon-sun (0-6)
#list of bad shifts [day, shift]
forbidden_shifts = [(d, s) for d in all_days for s in all_shifts if ((d % 7) >= 0 and (d % 7) < 5) and s == 1]

obj_int_vars = []
obj_int_coeffs = []
obj_bool_vars = []
obj_bool_coeffs = []

def generate_shifts():
    model = cp_model.CpModel()

    shifts = {}
    for n in all_members:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d, s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))


    # Each shift is assigned to exactly one member in .
    for d in all_days:
        for s in all_shifts:
            # if (d, s) not in forbidden_shifts:
            model.Add(sum(shifts[(n, d, s)] for n in all_members) == 1)

    # Each member works at most one shift per day.
    for n in all_members:
        for d in all_days:
            model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)

    # Try to distribute the shifts evenly, so that each member works
    # min_shifts_per_member shifts. If this is not possible, because the total
    # number of shifts is not divisible by the number of members, some members will
    # be assigned one more shift.
    # min_shifts_per_member = (num_shifts * num_days) // num_members
    # if num_shifts * num_days % num_members == 0:
    #     max_shifts_per_member = min_shifts_per_member
    # else:
    #     max_shifts_per_member = min_shifts_per_member + 1
    # for n in all_members:
    #     num_shifts_worked = []

    #     for d in all_days:
    #         for s in all_shifts:
    #             num_shifts_worked.append(shifts[(n, d, s)])
    #     model.Add(min_shifts_per_member <= sum(num_shifts_worked))
    #     model.Add(sum(num_shifts_worked) <= max_shifts_per_member)

    min_shift_per_member = (num_days) // num_members
    for n in all_members:
        for s in all_shifts:
            model.Add(sum(shifts[(n, d, s)] for d in all_days) >= min_shift_per_member)
            model.Add(sum(shifts[(n, d, s)] for d in all_days) <= min_shift_per_member + 1)

    # Penalized transitions
    for e in all_members:
        for d in range(num_days - 1):
            for s1 in all_shifts:
                for s2 in all_shifts:
                    transition = [
                        shifts[e, d, s1].Not(), shifts[e, d + 1, s2].Not()
                    ]
                    model.AddBoolOr(transition)

    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    if status == cp_model.OPTIMAL:
        print('Solution')
        days = {}
        for d in all_days:
            days[d] = {}
            for m in all_members:
                days[d][m] = "-"
                for s in all_shifts:
                    if (d, s) not in forbidden_shifts and solver.Value(shifts[(m, d, s)]):
                        days[d][m] = s
        display(pd.DataFrame(days))

generate_shifts()



Solution


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-,3,-,-,-,1,-,-,-,-,...,-,3,-,-,-,-,-,-,-,-
1,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,2,-
2,-,2,-,-,-,-,-,-,-,-,...,-,-,3,-,-,-,-,-,-,-
3,-,-,3,-,-,3,-,-,-,-,...,-,-,-,0,-,-,-,-,-,-
4,-,-,0,-,-,-,-,0,-,-,...,-,-,-,-,-,-,-,2,-,-
5,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6,-,-,2,-,2,-,-,-,-,-,...,3,-,-,-,0,-,-,-,-,-
7,2,-,-,-,-,-,1,-,-,-,...,-,-,-,-,-,-,-,-,-,-
8,-,-,-,2,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,3
9,-,-,-,-,3,-,-,-,-,-,...,-,-,-,-,-,-,0,-,-,-


In [ ]:
print('Solution')
        days = {}
        for d in all_days:
            days[d] = {}
            for m in all_members:
                days[d][m] = "-"
                for s in all_shifts:
                    if (d, s) not in forbidden_shifts and solver.Value(shifts[(m, d, s)]):
                        days[d][m] = s
        display(pd.DataFrame(days))

In [145]:

calc = (num_days * num_shifts) // num_members

print(calc)

6
